<a href="https://colab.research.google.com/github/noam1368/IRProject/blob/main/indexAnchorTextAndGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  WORKER_COUNT  PREEMPTIBLE_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-d98b  GCE       2                                       RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
pip install nltk==3.7

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# import findspark
# findspark.init()

In [6]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

ls: cannot access '/usr/lib/spark/jars/graph*': No such file or directory


In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [8]:
spark

In [9]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'corpus-big' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and b.name != "postings_gcp": #todo change when we want to run on the original corpuse:
        paths.append(full_path+b.name)



#just for the training
# bucket_name = 'training_little_corpus'   
# full_path = f"gs://{bucket_name}/"

# paths = []
# client = storage.Client()
# blobs = client.list_blobs(bucket_name)
# for b in blobs:
#     if b.name != '_SUCCESS' and b.name = 'indexTitle.parquet/': #todo change when we want to run on the original corpuse
#         paths.append(full_path+b.name)




***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [10]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("anchor_text", "id").rdd #this is with text becuase we doint the inverted index for the anchor_text in this file 

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [11]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [12]:
# doc_text_pairs.toDF().show()
parquetFile.show()

+-------+--------------------+--------------------+--------------------+
|     id|               title|                text|         anchor_text|
+-------+--------------------+--------------------+--------------------+
|4045403|Foster Air Force ...|'''Foster Air For...|[{1176764, Tactic...|
|4045413|     Torino Palavela|'''Palavela''', f...|[{77743, 2006 Win...|
|4045419|   Mad About the Boy|"'''Mad About the...|[{34028256, Joyce...|
|4045426|       Shayne Breuer|'''Shayne Breuer'...|[{1838386, Woodvi...|
|4045432|         Parantaka I|'''Parantaka Chol...|[{1511716, Aditya...|
|4045456|Arundel (UK Parli...|'''Arundel''' was...|[{4665376, Arunde...|
|4045466|     Andrew Martinez|'''Luis Andrew Ma...|[{4860, Berkeley,...|
|4045471|    Vancouver VooDoo|The '''Vancouver ...|[{32706, Vancouve...|
|4045479|     Invisible plane|The '''Invisible ...|[{2260539, Ross A...|
|4045516|    Shopping channel|'''Shopping chann...|[{592899, special...|
|4045519|      Turgay (river)|The '''Turgay''' ...|

In [13]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [14]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [15]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [16]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE
def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  # YOUR CODE HERE
  hash={}


  for word in tokens:
    if word in all_stopwords:
      continue
    if word in hash:
      hash[word] = (id, hash[word][1]+1)
    else:
      hash[word] = (id,1)


  return list(hash.items())


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key = lambda x:x[0])

def union_count(lst):
  dic={}
  for docs,value in lst:
    if docs in dic.keys():
      dic[docs]+=value
    else:
      dic[docs]=value
  return list(zip(dic.keys(),dic.values()))

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE

  return postings.map( lambda x: (x[0], len(x[1])) )



def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  # YOUR CODE HERE
  result = postings.map(lambda x: (token2bucket_id(x[0]),x)).groupByKey().map(lambda item: InvertedIndex.write_a_posting_list(item,bucket_name))  
  return result


In [ ]:
words = doc_text_pairs.flatMap(lambda y: map(lambda x: word_count(x[1],(y[1],x[0])),y[0]))
word_counts=words.flatMap(lambda x:x)
wordsone=word_counts.mapValues(lambda y:(y[0][1],y[1]))

postings=wordsone.groupByKey().mapValues(lambda x:union_count(sorted(list(x),key=lambda x:x[0])))
# postings_filtered = postings.filter(lambda x: len(x[1])>50)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()
posting_locs_list = partition_postings_and_write(postings).collect()

In [21]:
# # time the index creation time
# t_start = time()
# # word counts map
# word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
# postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# # filtering postings and calculate df
# # postings_filtered = postings.filter(lambda x: len(x[1])>50)
# w2df = calculate_df(postings)
# w2df_dict = w2df.collectAsMap()
# # partition posting lists and write out
# _ = partition_postings_and_write(postings).collect()
# index_const_time = time() - t_start

23/01/05 17:14:11 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 11.0 in stage 9.0 (TID 248) (cluster-d98b-w-1.c.assignment-3-370813.internal executor 7): TaskKilled (Stage cancelled)
23/01/05 17:14:11 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 12.0 in stage 9.0 (TID 249) (cluster-d98b-w-1.c.assignment-3-370813.internal executor 7): TaskKilled (Stage cancelled)
23/01/05 17:14:12 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 9.1 in stage 9.0 (TID 246) (cluster-d98b-w-0.c.assignment-3-370813.internal executor 6): TaskKilled (Stage cancelled)
23/01/05 17:14:12 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 10.0 in stage 9.0 (TID 247) (cluster-d98b-w-0.c.assignment-3-370813.internal executor 6): TaskKilled (Stage cancelled)
23/01/05 17:14:12 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 7.0 in stage 11.0 (TID 258) (cluster-d98b-w-1.c.assignment-3-370813.internal executor 1): org.apache.spark.api.python.PythonException: Traceback 

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 11.0 failed 4 times, most recent failure: Lost task 7.3 in stage 11.0 (TID 269) (cluster-d98b-w-0.c.assignment-3-370813.internal executor 5): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 594, in process
    out_iter = func(split_index, iterator)
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 2916, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark/python/pyspark/rdd.py", line 418, in func
    return f(iterator)
  File "/usr/lib/spar

23/01/05 17:14:17 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 13.1 in stage 11.0 (TID 276) (cluster-d98b-w-1.c.assignment-3-370813.internal executor 7): TaskKilled (Stage cancelled)
23/01/05 17:14:17 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 12.1 in stage 11.0 (TID 275) (cluster-d98b-w-1.c.assignment-3-370813.internal executor 7): TaskKilled (Stage cancelled)


In [18]:
# test index construction time
assert index_const_time < 60*120

NameError: name 'index_const_time' is not defined

In [19]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs('corpus-big', prefix='postings_gcp'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [20]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'indexAnchors')
# upload to gs
index_src = "indexAnchors.pkl"
index_dst = f'gs://indexes-my-proj/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://indexAnchors.pkl [Content-Type=application/octet-stream]...
| [1 files][ 95.0 MiB/ 95.0 MiB]                                                
Operation completed over 1 objects/95.0 MiB.                                     


In [26]:
print(inverted.posting_locs['min'])

[('102_002.bin', 1884768)]


# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [27]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and 
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the 
      second entry is the destination page id. No duplicates should be present. 
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph 
      created by the wikipedia links. No duplicates should be present. 
  '''
  # YOUR CODE HERE
  v1 = pages.map(lambda x:x[0])
  edges_not_distinct = pages.flatMap( lambda x: (map(lambda y: (x[0],y[0]),x[1])))
  edges = edges_not_distinct.distinct()
  v2 = edges.map(lambda v:v[1])
  vertices = v1.union(v2).distinct().map(lambda v: [v]) 
  # for p in edges_not_distinct.collect():
  #   print(p)
  return edges, vertices

In [ ]:
# t_start = time()
# pages_links = spark.read.parquet("gs://wikidata_preprocessed/*").select("id", "anchor_text").rdd
pages_links = parquetFile.select("id","anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
# pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 60*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

14.28 GiB    gs://wikidata_preprocessed


In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

5.92 GiB     gs://209206846/postings_gcp


In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0 
print(f'I used {cost} USD credit during the course of this assignment')

I used 0 USD credit during the course of this assignment


**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag

AssertionError: 